In [1]:
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

In [2]:
import json
import numpy as np
from typing import Any, Dict, List, Union, Optional
from tensorflow.keras.preprocessing import image

# Import model Facenet512
from facenetlib.Facenet_standalone import InceptionResNetV1

# Import RetinaFace detector
from retina import detect_faces, create_retinaface_model

# Import preprocessing utils DeepFace
from facenetlib import preprocessing, image_utils

In [3]:
# =========================
# Load model embedding
# =========================
model = InceptionResNetV1(dimension=512)
model.load_weights("facenetlib/facenet512_weights.h5")  # sesuaikan path jika perlu
print("✅ Model Facenet512 berhasil dimuat.")




✅ Model Facenet512 berhasil dimuat.


In [4]:
# =========================
# Load model detect
# =========================
modeld = create_retinaface_model()


Building RetinaFace model architecture...
Loading weights into model...
Weights loaded successfully!


In [6]:

# =========================
# Represent function
# mirip representation.py
# =========================
def represent(
    img_path: Union[str, np.ndarray],
    enforce_detection: bool = True,
    align: bool = True,
    expand_percentage: int = 0,
    normalization: str = "base",
    threshold: float = 0.9
) -> List[Dict[str, Any]]:
    """
    Ekstraksi embedding wajah menggunakan RetinaFace sebagai detektor.
    Output format mirip DeepFace `represent`.
    """
    resp_objs = []

    # Load gambar (RGB)
    img, _ = image_utils.load_image(img_path)
    if img.shape[2] == 3:
        img_bgr = img[:, :, ::-1]  # ubah urutan channel
    else:
        img_bgr = img
    # Deteksi wajah
    detections = detect_faces(
        img_path=img_bgr,
        threshold=threshold,
        model=modeld
    )
    
    if isinstance(detections, dict) and all(isinstance(v, dict) for v in detections.values()):
        detections = list(detections.values())

    # Pastikan list of dict
    if not isinstance(detections, list) or not all(isinstance(d, dict) for d in detections):
        raise ValueError(f"Format hasil detect_faces tidak valid: {type(detections)}")

    if enforce_detection and len(detections) == 0:
        raise ValueError(f"Tidak ada wajah terdeteksi pada {img_path}")

    for det in detections:
        coords = det.get("facial_area")
        if isinstance(coords, list) and len(coords) == 4:
            x1, y1, x2, y2 = coords
            facial_area = {
                "x": int(x1),
                "y": int(y1),
                "w": int(x2 - x1),
                "h": int(y2 - y1)
            }
        elif isinstance(coords, dict):
            facial_area = coords
        else:
            raise ValueError(f"Format facial_area tidak valid: {coords}")

        confidence = det.get("confidence") or det.get("score", 0.0)

        # Crop wajah
        face_img = img[
            facial_area["y"]: facial_area["y"] + facial_area["h"],
            facial_area["x"]: facial_area["x"] + facial_area["w"]
        ]
        # Resize untuk Facenet
        face_img = preprocessing.resize_image(face_img, target_size=(160, 160))

        # Normalisasi sesuai Facenet
        face_img = preprocessing.normalize_input(face_img, normalization=normalization)

        # Pastikan shape input (1, 160, 160, 3)
        if face_img.ndim == 3:
            face_img = np.expand_dims(face_img, axis=0)
        elif face_img.ndim == 4 and face_img.shape[0] == 1:
            pass  # sudah batch dim
        else:
            raise ValueError(f"Bentuk input tidak dikenali: {face_img.shape}")

        # Forward pass
        embedding = model.predict(face_img, verbose=0)[0]

        resp_objs.append({
            "embedding": embedding.astype(float).tolist(),
            "facial_area": facial_area,
            "face_confidence": confidence
        })

    return resp_objs



# =========================
# Contoh penggunaan
# =========================
if __name__ == "__main__":
    input_folder = "images"  # ganti dengan path folder kamu
    output_json = "embedding/embeddings.json"
    embeddings_dict = {}

    for filename in os.listdir(input_folder):
        if filename.lower().endswith((".png", ".jpg", ".jpeg")):
            img_path = os.path.join(input_folder, filename)
            print(f"📷 Memproses: {filename}")
            try:
                results = represent(img_path)
                # Ambil embedding wajah pertama (kalau ada banyak wajah bisa disesuaikan)
                embeddings_dict[filename] = results[0]["embedding"] if results else None
            except Exception as e:
                print(f"⚠️ Gagal memproses {filename}: {e}")

    with open(output_json, "w") as f:
        json.dump(embeddings_dict, f, indent=2)

    print(f"✅ Embeddings disimpan di {output_json}")


📷 Memproses: IMG_2042.jpg
📷 Memproses: WhatsApp Image 2025-07-30 at 16.57.34.jpeg
📷 Memproses: WhatsApp Image 2025-07-30 at 16.57.37.jpeg
📷 Memproses: WhatsApp Image 2025-07-30 at 16.57.47.jpeg
📷 Memproses: WhatsApp Image 2025-07-30 at 16.57.48.jpeg
✅ Embeddings disimpan di embedding/embeddings.json
